In [8]:
import requests
from bs4 import BeautifulSoup
import re
import json
import time
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor, wait

In [43]:
def worker(critic_link):
    
    resp = requests.get(critic_link)
    soup = BeautifulSoup(resp.text, 'lxml')
    
    movie_dict_list = []
    
    splitter_list = [word.text for word in soup.select_one("div.inner").select("p")]     #先確定資訊頁和內容頁分隔在哪一行
    count = 1
    for word in splitter_list:
        if "找不到連結！" not in word:
            if "主演" in word or "演員" in word or "配音" in word:
                splitter = count
                break
            elif "導演" in word or "執行導演" in word:
                splitter = count
                break
            count+=1
        else:
            break
    info = "".join(splitter_list[0:splitter])   #找出內文前的資訊
    
    print(critic_link)
    movie_dict = {}    
    movie_dict['author']      = re.findall('<p>文:(.+)</p>', resp.text)
    movie_dict['name']        = info.split('\n')[0]
    movie_dict['critic']      = "".join(splitter_list[splitter+1:])                   #找出評論內文同時加進字典

    movie_dict_list.append({ 
                            'movietitle' : movie_dict['name'],
                            'post'       : movie_dict['critic'],
                            'author'     : movie_dict['author']
    })
   

In [44]:
def link_crawler(titles_num):
    
    critics_link_urls = []
    
    for i in range(0, len(titles_num)):
        url_part = soup.select('div.inner > p > a')[i]['href']
        critics_url = DOMAIN + url_part
        if 'hkmreviews' in url_part and 'en' not in url_part:         #只抓中文影評
            critics_link_urls.append(critics_url)                     #至此抓到所有網頁連結
    print(critics_link_urls)
    
    for critic_link in critics_link_urls:
        worker(critic_link)            

In [45]:
if __name__ == "__main__":
    
    tStart = time.time()
    DOMAIN = "http://www.cinespot.com"
    URL = 'http://www.cinespot.com/hkmreviews'
    resp = requests.get(URL)
    soup = BeautifulSoup(resp.text, 'lxml')
    
    titles_num = soup.select('div.inner > p > a')
    link_crawler(titles_num)
    
    with concurrent.futures.ThreadPoolExecutor(80) as executor:
        executor.map(worker, critic_link)
    
    with open('C:\\Users\\Java\\ETL\\movie\\Cinespot\\cinespot.json', 'w') as f:
        jd = json.dumps(movie_dict_list, ensure_ascii=False, indent=4)
        jd.encode('utf-8')
        f.write(jd)
        
    tEnd = time.time()
    print(tEnd - tStart)

['http://www.cinespot.com/hkmreviews/黑白迷宮', 'http://www.cinespot.com/hkmreviews/西謊極落之太爆太子太空艙', 'http://www.cinespot.com/hkmreviews/追捕', 'http://www.cinespot.com/hkmreviews/追捕-2', 'http://www.cinespot.com/hkmreviews/夜色撩人', 'http://www.cinespot.com/hkmreviews/雖然媽媽說我不可以嫁去日本', 'http://www.cinespot.com/hkmreviews/破局', 'http://www.cinespot.com/hkmreviews/翻滾吧男人', 'http://www.cinespot.com/hkmreviews/美好的意外', 'http://www.cinespot.com/hkmreviews/追龍', 'http://www.cinespot.com/hkmreviews/京城81號2', 'http://www.cinespot.com/hkmreviews/紅衣小女孩2', 'http://www.cinespot.com/hkmreviews/報告老師-怪怪怪怪物', 'http://www.cinespot.com/hkmreviews/台北物語', 'http://www.cinespot.com/hkmreviews/今晚打喪屍-2', 'http://www.cinespot.com/hkmreviews/今晚打喪屍', 'http://www.cinespot.com/hkmreviews/春蠶', 'http://www.cinespot.com/hkmreviews/神女', 'http://www.cinespot.com/hkmreviews/大路', 'http://www.cinespot.com/hkmreviews/俠女', 'http://www.cinespot.com/hkmreviews/愛奴', 'http://www.cinespot.com/hkmreviews/蝶變', 'http://www.cinespot.com/hkmreviews/鎗火

http://www.cinespot.com/hkmreviews/黑白迷宮
http://www.cinespot.com/hkmreviews/西謊極落之太爆太子太空艙
http://www.cinespot.com/hkmreviews/追捕
http://www.cinespot.com/hkmreviews/追捕-2
http://www.cinespot.com/hkmreviews/夜色撩人
http://www.cinespot.com/hkmreviews/雖然媽媽說我不可以嫁去日本
http://www.cinespot.com/hkmreviews/破局
http://www.cinespot.com/hkmreviews/翻滾吧男人
http://www.cinespot.com/hkmreviews/美好的意外
http://www.cinespot.com/hkmreviews/追龍
http://www.cinespot.com/hkmreviews/京城81號2
http://www.cinespot.com/hkmreviews/紅衣小女孩2
http://www.cinespot.com/hkmreviews/報告老師-怪怪怪怪物
http://www.cinespot.com/hkmreviews/台北物語
http://www.cinespot.com/hkmreviews/今晚打喪屍-2
http://www.cinespot.com/hkmreviews/今晚打喪屍
http://www.cinespot.com/hkmreviews/春蠶
http://www.cinespot.com/hkmreviews/神女
http://www.cinespot.com/hkmreviews/大路
http://www.cinespot.com/hkmreviews/俠女
http://www.cinespot.com/hkmreviews/愛奴
http://www.cinespot.com/hkmreviews/蝶變
http://www.cinespot.com/hkmreviews/鎗火-2
http://www.cinespot.com/hkmreviews/站台
http://www.cinespot.com/hkm

http://www.cinespot.com/hkmreviews/九層妖塔
http://www.cinespot.com/hkmreviews/一步之遙
http://www.cinespot.com/hkmreviews/一生一世
http://www.cinespot.com/hkmreviews/白日焰火
http://www.cinespot.com/hkmreviews/私人訂制-2
http://www.cinespot.com/hkmreviews/私人訂制
http://www.cinespot.com/hkmreviews/不二神探
http://www.cinespot.com/hkmreviews/一九四二
http://www.cinespot.com/hkmreviews/一九四二-2
http://www.cinespot.com/hkmreviews/危險關係
http://www.cinespot.com/hkmreviews/百萬巨鱷
http://www.cinespot.com/hkmreviews/形影不離
http://www.cinespot.com/hkmreviews/夜店詭談
http://www.cinespot.com/hkmreviews/極速天使
http://www.cinespot.com/hkmreviews/英雄喋血
http://www.cinespot.com/hkmreviews/白蛇傳說-2
http://www.cinespot.com/hkmreviews/白蛇傳說
http://www.cinespot.com/hkmreviews/辛亥革命
http://www.cinespot.com/hkmreviews/辛亥革命-2
http://www.cinespot.com/hkmreviews/雪花秘扇
http://www.cinespot.com/hkmreviews/藏獒多吉
http://www.cinespot.com/hkmreviews/隱婚男女
http://www.cinespot.com/hkmreviews/建黨偉業
http://www.cinespot.com/hkmreviews/致命請柬
http://www.cinespot.com/hkmrevie

UnboundLocalError: local variable 'splitter' referenced before assignment